In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet34
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image

print("Environment setup is done")

Environment setup is done


In [2]:
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256

# 数据集路径
train_path = 'autodl-tmp/ANIMAL-10N/training'
test_path = 'autodl-tmp/ANIMAL-10N/testing'

# 读取图像文件和标签
def load_images_labels(file_path):
    images, labels = [], []
    images_file = os.listdir(file_path)
    for image_file in images_file:
        image_path = os.path.join(file_path, image_file)
        label = int(image_file[0])  # 文件名首字母为标签
        image = Image.open(image_path).convert('RGB')
        images.append(image)
        labels.append(label)
    return images, labels

# 加载训练和测试数据
train_images, train_labels = load_images_labels(train_path)
test_images, test_labels = load_images_labels(test_path)

# 自定义 Dataset 类
class AnimalDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# 数据增强（训练集和测试集）
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 创建数据加载器
test_train_set = AnimalDataset(images=test_images, labels=test_labels, transform=transforms_train)
test_train_loader = DataLoader(test_train_set, batch_size=batch_size, shuffle=True, num_workers=4)

train_test_set = AnimalDataset(images=train_images, labels=train_labels, transform=transforms_test)
train_test_loader = DataLoader(train_test_set, batch_size=batch_size, shuffle=False, num_workers=4)

In [3]:
# 定义 ResNet 模型
def initialize_model():
    model = resnet34(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 10)  # 数据集类别数为10
    return model.to(device)

# 定义损失函数和优化器
def initialize_optimizer(model):
    optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    return optimizer, scheduler

In [4]:
# 训练函数
def train(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets).mean()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    avg_loss = running_loss / len(loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

# 测试函数
def test(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets).mean()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    avg_loss = running_loss / len(loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

# 创建文件夹
os.makedirs('log', exist_ok=True)
os.makedirs('weight', exist_ok=True)

In [ ]:
print(f"-----------------直接训练开始-----------------")
    
model = initialize_model()
optimizer, scheduler = initialize_optimizer(model)
criterion = nn.CrossEntropyLoss(reduction='none')

train_losses, train_accuracies, test_losses, test_accuracies = [], [], [], []

# 直接训练
for epoch in range(100):
    train_loss, train_accuracy = train(model, train_test_loader, criterion, optimizer)  # 用测试集训练
    test_loss, test_accuracy = test(model, test_train_loader, criterion)  # 用训练集测试
    scheduler.step()
        
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(f'Epoch {epoch + 1} - Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%')
    if train_accuracy > 99.999:
        break

# 保存训练日志
pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Train Loss': train_losses,
    'Train Accuracy': train_accuracies,
    'Test Loss': test_losses,
    'Test Accuracy': test_accuracies
}).to_csv(f'log/animal_train_on_test_data.csv', index=False)
    
torch.save(model.state_dict(), f'weight/resnet34_animal_trained_on_test.pth')
print(f"训练过程已保存到 'log/animal_train_on_test_data.csv'，模型权重保存为 'weight/resnet34_animal_trained_on_test.pth'")

-----------------直接训练开始-----------------
Epoch 1 - Train Loss: 13.0470, Train Acc: 61.59%, Test Loss: 45.4810, Test Acc: 9.50%
Epoch 2 - Train Loss: 20.6834, Train Acc: 34.90%, Test Loss: 43.2687, Test Acc: 9.50%
Epoch 3 - Train Loss: 20.5145, Train Acc: 34.98%, Test Loss: 43.1418, Test Acc: 9.50%
Epoch 4 - Train Loss: 20.3516, Train Acc: 35.41%, Test Loss: 42.7341, Test Acc: 9.50%
Epoch 5 - Train Loss: 20.3068, Train Acc: 35.41%, Test Loss: 42.6315, Test Acc: 9.50%
Epoch 6 - Train Loss: 20.3208, Train Acc: 35.41%, Test Loss: 42.8128, Test Acc: 9.50%
Epoch 7 - Train Loss: 20.2896, Train Acc: 35.41%, Test Loss: 42.6273, Test Acc: 9.50%
Epoch 8 - Train Loss: 20.3028, Train Acc: 35.41%, Test Loss: 42.7455, Test Acc: 9.50%
Epoch 9 - Train Loss: 20.2990, Train Acc: 35.41%, Test Loss: 42.6950, Test Acc: 9.50%
Epoch 10 - Train Loss: 50.4027, Train Acc: 73.85%, Test Loss: 1386.1429, Test Acc: 9.50%
